In [7]:
# Workflow and process 

In [8]:
# Steps
# 1. import the csv, clean it up, geocode it and get the coordinates 
# 2. use mapbox to build the maps and adobe
#     2a. choose a colour pallete and font family stuff 
#     - canada wide
#     - east coast highlight 
#     - halifax highlight (adobe) 
#     - common highlight (adobe?)
#     - facebook marketplace highlight (adobe) 
#     - survey respondents - where they live, how they call it
#     - flower streets and dartmouth 
#     - north end 
# 3. build out in datawrapper an interactive click of people's responses and then what they said for their neighbourhood 
#     - can this show frequency or closeness at all?
# 4. layer from halifax geo map or something????
# 5. build out html scrolly 
# 6. learn how to publish 

In [9]:
import pandas as pd
import numpy as np
import warnings, os
import re

In [10]:
#import the data in order to clean it up and geocode it 

In [11]:
data = pd.read_csv('data-jun-27.csv')

In [12]:
data.head()

,Timestamp,What is your address?,How do you call or refer to the neighbourhood or area where you live?,Is there a secondary way you refer to where you live? Share here,Do you have further thoughts about how you refer to where you live?,Are you interesting in telling me more about this topic?,"If yes, please provide your name and an email address I can reach you at."
0,21/06/2025 09:28:19,2351 Maynard Street,The north end,Halifax,I have never said HRM in my life,Yes,tara.thorne@gmail.com
1,21/06/2025 09:29:12,Ford St,Fairview,The Peninsula's armpit,No,Yes,Calebschriver@gmail.com
2,21/06/2025 09:29:58,76 main ave,Fairview,Halifax,I don't but I'm happy to talk to you about it,Yes,Kdwakeman@gmail.com
3,21/06/2025 09:44:53,"53 Tulip Street, Dartmouth",Flower Streets,Dartmouth,Apparently it was known historically as Austen...,Yes,"Jacob Boon, jboon@thetyee"
4,21/06/2025 09:46:08,36 Hansens,waverley,my road,I'm in a secluded lil area w maybe not a Lot o...,NaN,NaN


In [13]:
data.columns


Index(['Timestamp', 'What is your address? ',
       'How do you call or refer to the neighbourhood or area where you live?',
       'Is there a secondary way you refer to where you live? Share here',
       'Do you have further thoughts about how you refer to where you live?',
       'Are you interesting in telling me more about this topic?',
       'If yes, please provide your name and an email address I can reach you at. '],
      dtype='object')

In [14]:
#rename the columns that are being kept
data_cleaned = data.rename(columns={"What is your address? ":"Address", "How do you call or refer to the neighbourhood or area where you live?":"Name", "Is there a secondary way you refer to where you live? Share here":"Secondary name" })

In [15]:
#drop columns I dont need 
data_cleaned.drop(columns= ['Timestamp','Do you have further thoughts about how you refer to where you live?',
       'Are you interesting in telling me more about this topic?',
       'If yes, please provide your name and an email address I can reach you at. '], inplace=True)

In [16]:
data_cleaned.head()

,Address,Name,Secondary name
0,2351 Maynard Street,The north end,Halifax
1,Ford St,Fairview,The Peninsula's armpit
2,76 main ave,Fairview,Halifax
3,"53 Tulip Street, Dartmouth",Flower Streets,Dartmouth
4,36 Hansens,waverley,my road


In [17]:
data_cleaned["Name"].nunique()

24

In [18]:
data_cleaned["Name"].unique()

array(['The north end', 'Fairview', 'Flower Streets', 'waverley',
       'Downtown ', 'North end ', 'north end ',
       'Dartmouth, Portland Street area, like toward Cole harbour way. ',
       'Downtown Dartmouth', 'Larry Uteck', 'Bedford ', 'West end',
       'North End', 'Downtown Dartmouth ', 'Sullivan’s Pond',
       'The North End ', 'Quinpool', 'Bedford',
       'I usually say "Central Halifax" ', 'the South End',
       'West bedford', 'The West End', 'Larry Utech',
       '(Bottom of the) North End'], dtype=object)

In [19]:
data_cleaned[data_cleaned['Name'].str.contains("North end", na=False)]

,Address,Name,Secondary name
6,2494 John Street,North end,“Near Agricola” or “near the commons”
29,6143 Charles Street,North end,near the common


In [20]:
#get rid of spaces, get rid of caps 
data_cleaned['Area'] = data_cleaned['Name'].str.lower().str.strip()

In [21]:
data_cleaned

,Address,Name,Secondary name,Area
0,2351 Maynard Street,The north end,Halifax,the north end
1,Ford St,Fairview,The Peninsula's armpit,fairview
2,76 main ave,Fairview,Halifax,fairview
3,"53 Tulip Street, Dartmouth",Flower Streets,Dartmouth,flower streets
4,36 Hansens,waverley,my road,waverley
5,1583 Hollis St,Downtown,NaN,downtown
6,2494 John Street,North end,“Near Agricola” or “near the commons”,north end
7,2494 John St,north end,off agricola,north end
8,"18 Hamlet Lane, Dartmouth, NS.","Dartmouth, Portland Street area, like toward C...","Dartmouth, 5 minutes to Cole Harbour.","dartmouth, portland street area, like toward c..."
9,70 Ochterloney Street,Downtown Dartmouth,NaN,downtown dartmouth


In [22]:
#creating a function to sort the answers into larger, standardized buckets (there must be a simpler way, but I like that I can understand what's happening here) 
def generalize_area(name):
    if 'north end' in name:
        return 'north end'
    elif 'south end' in name:
        return 'south end'
    elif 'downtown dartmouth' in name:
        return 'downtown dartmouth'
    elif 'downtown' in name:
        return 'downtown'
    elif 'west end' in name:
        return 'west end'
    elif 'larry uteck' in name or 'larry utech' in name:
        return 'larry uteck'
    elif 'bedford' in name:
        return 'bedford'
    elif 'fairview' in name:
        return 'fairview'
    elif 'waverley' in name:
        return 'waverley'
    elif 'quinpool' in name:
        return 'west end' #this one was manual
    elif 'central halifax' in name:
        return 'central halifax'
    elif 'sullivan' in name and 'pond' in name:
        return "dartmouth" #this one was manual
    elif 'flower streets' in name:
        return 'dartmouth' #this one was manual
    elif 'dartmouth' in name:
        return 'dartmouth'
    else:
        return 'unknown'


In [23]:
#applying the above function to the Name_cleaned column, giving the bigger buckets
data_cleaned['Area'] = data_cleaned['Area'].apply(generalize_area)

In [24]:
data_cleaned.head()

,Address,Name,Secondary name,Area
0,2351 Maynard Street,The north end,Halifax,north end
1,Ford St,Fairview,The Peninsula's armpit,fairview
2,76 main ave,Fairview,Halifax,fairview
3,"53 Tulip Street, Dartmouth",Flower Streets,Dartmouth,dartmouth
4,36 Hansens,waverley,my road,waverley


In [25]:
data_cleaned['Area'].value_counts()

Area
north end             7
fairview              4
downtown dartmouth    4
dartmouth             3
bedford               3
west end              3
larry uteck           2
waverley              1
downtown              1
central halifax       1
south end             1
Name: count, dtype: int64

In [26]:
#I'm satisfied with those buckets for further storytelling, now I'll clean the addresses so I can get the geocoordinates for everyone's addresses if I can

In [27]:
address_parts = data_cleaned['Address'].str.split(',', expand=True)

In [28]:
data_cleaned['Street'] = address_parts[0].str.strip()
data_cleaned['City'] = address_parts[1].str.strip() if 1 in address_parts.columns else ''
data_cleaned['Province'] = address_parts[2].str.strip() if 2 in address_parts.columns else ''

In [29]:
data_cleaned['City'] = data_cleaned['City'].replace('', pd.NA)
data_cleaned['City'] = data_cleaned['City'].fillna('Halifax')
data_cleaned['Province'] = 'NS'
data_cleaned['Country'] = 'Canada'

In [30]:
data_cleaned['Address_cleaned'] = data_cleaned['Address'].astype(str).str.strip()

In [31]:
data_cleaned.head()

,Address,Name,Secondary name,Area,Street,City,Province,Country,Address_cleaned
0,2351 Maynard Street,The north end,Halifax,north end,2351 Maynard Street,Halifax,NS,Canada,2351 Maynard Street
1,Ford St,Fairview,The Peninsula's armpit,fairview,Ford St,Halifax,NS,Canada,Ford St
2,76 main ave,Fairview,Halifax,fairview,76 main ave,Halifax,NS,Canada,76 main ave
3,"53 Tulip Street, Dartmouth",Flower Streets,Dartmouth,dartmouth,53 Tulip Street,Dartmouth,NS,Canada,"53 Tulip Street, Dartmouth"
4,36 Hansens,waverley,my road,waverley,36 Hansens,Halifax,NS,Canada,36 Hansens


In [32]:
# Combine into a single full address column
data_cleaned['Full_address'] = data_cleaned['Street'] + ', ' + data_cleaned['City'] + ', ' + data_cleaned['Province'] + ', ' + data_cleaned['Country']


In [33]:
data_cleaned.head()

,Address,Name,Secondary name,Area,Street,City,Province,Country,Address_cleaned,Full_address
0,2351 Maynard Street,The north end,Halifax,north end,2351 Maynard Street,Halifax,NS,Canada,2351 Maynard Street,"2351 Maynard Street, Halifax, NS, Canada"
1,Ford St,Fairview,The Peninsula's armpit,fairview,Ford St,Halifax,NS,Canada,Ford St,"Ford St, Halifax, NS, Canada"
2,76 main ave,Fairview,Halifax,fairview,76 main ave,Halifax,NS,Canada,76 main ave,"76 main ave, Halifax, NS, Canada"
3,"53 Tulip Street, Dartmouth",Flower Streets,Dartmouth,dartmouth,53 Tulip Street,Dartmouth,NS,Canada,"53 Tulip Street, Dartmouth","53 Tulip Street, Dartmouth, NS, Canada"
4,36 Hansens,waverley,my road,waverley,36 Hansens,Halifax,NS,Canada,36 Hansens,"36 Hansens, Halifax, NS, Canada"


In [34]:
# Convert to list for geocoding
address_list = data_cleaned['Full_address'].tolist()

In [35]:
street_list = data_cleaned['Street'].tolist()

In [36]:
city_list = data_cleaned['City'].tolist()

In [37]:
full_addresses_df = data_cleaned['Full_address']

In [38]:
full_addresses_df.to_csv('Full_address.csv', index=False)

In [39]:
len(address_list)

30

In [40]:
print(data_cleaned['Full_address'])

0              2351 Maynard Street, Halifax, NS, Canada
1                          Ford St, Halifax, NS, Canada
2                      76 main ave, Halifax, NS, Canada
3                53 Tulip Street, Dartmouth, NS, Canada
4                       36 Hansens, Halifax, NS, Canada
5                   1583 Hollis St, Halifax, NS, Canada
6                 2494 John Street, Halifax, NS, Canada
7                     2494 John St, Halifax, NS, Canada
8                 18 Hamlet Lane, Dartmouth, NS, Canada
9            70 Ochterloney Street, Halifax, NS, Canada
10            635 Larry Uteck blvd, Halifax, NS, Canada
11            Ochterloney Street, Dartmouth, NS, Canada
12        23 bridge street Bedford, Halifax, NS, Canada
13              3208 Connaught Ave, Halifax, NS, Canada
14                   5274 Glebe St, Halifax, NS, Canada
15    33 Prince Albert Rd. Dartmouth, Halifax, NS, C...
16                33 Prince Albert, Halifax, NS, Canada
17               84 Crichton Ave, Dartmouth, NS,

In [41]:
# #now I will see if these are clean enough to be geocoded
# import requests
# params = {
#     'api_key': '8eeb81e1ae1860eb9e8a785155ef86e1b76171e',
#     'q' : address_list
#     # 'street': street_list,
#     # 'city': city_list,
#     # 'state': "NS",
#     # 'country': 'Canada'
# }
# response = requests.get('https://api.geocod.io/v1.7/geocode', params=params)

# if response.status_code != 200:
#     raise Exception(f"Geocodio API error: {response.status_code} - {response.text}")

# results = response.json()['results']


In [42]:
# len(results)

In [43]:
from geopy.geocoders import Nominatim
from time import sleep
import pandas as pd

In [44]:
# Sample data #got this code from TA Ananya
df = pd.read_csv('Full_address.csv')
# (and then make sure your csv has the full_address of borough.park name) 
# Set up geocoder
geolocator = Nominatim(user_agent="ananya-geopy-lookup")
# Function to get location
def geocode_address(address):
    try:
        location = geolocator.geocode(address)
        sleep(1)  # Respectful delay to avoid rate limiting
        return location
    except:
        return None
# Geocode each address
df['location'] = df['Full_address'].apply(geocode_address)
# Now extract latitude and longitude
df['latitude'] = df['location'].apply(lambda loc: loc.latitude if loc else None)
df['longitude'] = df['location'].apply(lambda loc: loc.longitude if loc else None)
# df = df.drop(columns='location')  # Optional: remove location column
df

,Full_address,location,latitude,longitude
0,"2351 Maynard Street, Halifax, NS, Canada","(2351, Maynard Street, Downtown Halifax, Halif...",44.653145,-63.587796
1,"Ford St, Halifax, NS, Canada","(Ford Street, Halifax, Halifax Regional Munici...",44.661329,-63.633025
2,"76 main ave, Halifax, NS, Canada","(76, Main Avenue, Fairview Estates, Halifax, H...",44.658943,-63.635353
3,"53 Tulip Street, Dartmouth, NS, Canada","(53, Tulip Street, Austinville, Downtown Dartm...",44.673424,-63.566665
4,"36 Hansens, Halifax, NS, Canada","(36, Hansens Lane, Waverley, Halifax Regional ...",44.788636,-63.601470
5,"1583 Hollis St, Halifax, NS, Canada","(The Maple, 1583, Hollis Street, Downtown Hali...",44.646304,-63.571850
6,"2494 John Street, Halifax, NS, Canada","(2494, John Street, Halifax, Halifax Regional ...",44.653697,-63.592728
7,"2494 John St, Halifax, NS, Canada","(2494, John Street, Halifax, Halifax Regional ...",44.653697,-63.592728
8,"18 Hamlet Lane, Dartmouth, NS, Canada","(18, Hamlet Lane, Evergreen Place, Dartmouth, ...",44.668395,-63.538061
9,"70 Ochterloney Street, Halifax, NS, Canada","(Founders’ Corner, 70, Ochterloney Street, Che...",44.666960,-63.567585


In [45]:
merged_df = df.merge(data_cleaned[['Full_address', 'Area']], on='Full_address', how='left')

In [46]:
merged_df.columns

Index(['Full_address', 'location', 'latitude', 'longitude', 'Area'], dtype='object')

In [47]:
merged_df[['Full_address', 'Area', 'latitude', 'longitude']].to_csv('map_points.csv', index=False)
#now to take this .csv to Map Box and make my maps! 

In [49]:
new_merged_df = merged_df.drop('Full_address', axis=1)


In [50]:
new_merged_df = new_merged_df[['latitude', 'longitude', 'Area']]

In [51]:
new_merged_df.head()

,latitude,longitude,Area
0,44.653145,-63.587796,north end
1,44.661329,-63.633025,fairview
2,44.658943,-63.635353,fairview
3,44.673424,-63.566665,dartmouth
4,44.788636,-63.601470,waverley


In [52]:
new_merged_df.rename(columns={'Area': 'Name'}, inplace=True)


In [55]:
new_merged_df[['latitude', 'longitude', 'Name']].to_csv('map_points_2.csv', index=False)
#now to take this .csv to Map Box and make my maps! 

In [3]:
!pip install geojson


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import csv
import geojson

In [58]:

from geojson import Feature, FeatureCollection, Point

features = []
with open('map_points_2.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        latitude_str, longitude_str, area = row[:3]

        # Skip if lat or lon is missing
        if not latitude_str.strip() or not longitude_str.strip():
            continue

        try:
            latitude = float(latitude_str)
            longitude = float(longitude_str)
        except ValueError:
            continue  # Skip if conversion fails

    # for latitude, longitude, area in reader:
    #     latitude, longitude = map(float, (latitude, longitude))
            features.append(
                Feature(
                    geometry = Point((longitude, latitude)),
                    properties = {
                        'Name': Name
                }
            )
        )

collection = FeatureCollection(features)
with open("GeoObs.json", "w") as f:
    f.write('%s' % collection)

In [59]:
collection

{"features": [], "type": "FeatureCollection"}

In [60]:
for row in reader:
    print("Row:", row)  # Add this to inspect what the script sees
    latitude_str, longitude_str, area = row[:3]
    

ValueError: I/O operation on closed file.

In [63]:
import csv
from geojson import Feature, FeatureCollection, Point

features = []

with open('map_points_2.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    header = next(reader)
    print("Header:", header)

    for row in reader:
        print("Row:", row)  # ✅ This must be *inside* the 'with' block

        latitude_str, longitude_str, Name = row[:3]

        if not latitude_str.strip() or not longitude_str.strip():
            print("Skipping row with blank lat/lon.")
            continue

        try:
            latitude = float(latitude_str)
            longitude = float(longitude_str)
        except ValueError:
            print(f"Skipping invalid lat/lon: {latitude_str}, {longitude_str}")
            continue

        print(f"Adding point: ({latitude}, {longitude}) -> {area}")
        features.append(
            Feature(
                geometry=Point((longitude, latitude)),
                properties={'Name': Name}
            )
        )

collection = FeatureCollection(features)

with open("GeoObs.json", "w") as f:
    f.write('%s' % collection)

Header: ['latitude', 'longitude', 'Name']
Row: ['44.6531452', '-63.5877963', 'north end']
Adding point: (44.6531452, -63.5877963) -> north end
Row: ['44.661329', '-63.633025', 'fairview']
Adding point: (44.661329, -63.633025) -> north end
Row: ['44.6589432', '-63.6353532', 'fairview']
Adding point: (44.6589432, -63.6353532) -> north end
Row: ['44.6734236', '-63.5666655', 'dartmouth']
Adding point: (44.6734236, -63.5666655) -> north end
Row: ['44.7886362', '-63.6014701', 'waverley']
Adding point: (44.7886362, -63.6014701) -> north end
Row: ['44.6463038', '-63.5718496', 'downtown']
Adding point: (44.6463038, -63.5718496) -> north end
Row: ['44.6536965', '-63.5927276', 'north end']
Adding point: (44.6536965, -63.5927276) -> north end
Row: ['44.6536965', '-63.5927276', 'north end']
Adding point: (44.6536965, -63.5927276) -> north end
Row: ['44.6683948', '-63.5380611', 'dartmouth']
Adding point: (44.6683948, -63.5380611) -> north end
Row: ['44.6669598', '-63.5675845', 'downtown dartmouth']


In [66]:
import csv
from geojson import Feature, FeatureCollection, Point

features = []

with open('map_points_2.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    header = next(reader)
    print("Header:", header)

    for row in reader:
        latitude_str, longitude_str, name = row[:3]

        if not latitude_str.strip() or not longitude_str.strip():
            continue

        try:
            latitude = float(latitude_str)
            longitude = float(longitude_str)
        except ValueError:
            continue

        name = name.title()

        features.append(
            Feature(
                geometry=Point((longitude, latitude)),
                properties={'Name': name}
            )
        )

collection = FeatureCollection(features)

with open("GeoObs.json", "w") as f:
    f.write('%s' % collection)


Header: ['latitude', 'longitude', 'Name']
